In [29]:
import cv2
import numpy as np

In [30]:
# OpenCV에서 카메라(웹캠)로부터 영상을 전달받아 처리하기 위해서는 VideoCapture 클래스를 사용
# cv2에 웹캠 연결
VideoSignal = cv2.VideoCapture(1)

In [31]:
# 학습된 네트워크 불러오기
YOLO_net = cv2.dnn.readNet("yolov3-tiny.weights","yolov3-tiny.cfg")

In [32]:
# YOLO NETWORK 재구성
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# 출력 레이어를 결정
layer_names = YOLO_net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in YOLO_net.getUnconnectedOutLayers()]

# 클래스의 갯수만큼 랜덤 RGB 배열을 생성
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [33]:
YOLO_net.getUnconnectedOutLayers()

array([[36],
       [48]], dtype=int32)

In [34]:
print(output_layers)
print(layer_names)


['yolo_16', 'yolo_23']
['conv_0', 'bn_0', 'relu_1', 'pool_1', 'conv_2', 'bn_2', 'relu_3', 'pool_3', 'conv_4', 'bn_4', 'relu_5', 'pool_5', 'conv_6', 'bn_6', 'relu_7', 'pool_7', 'conv_8', 'bn_8', 'relu_9', 'pool_9', 'conv_10', 'bn_10', 'relu_11', 'pool_11', 'conv_12', 'bn_12', 'relu_13', 'conv_13', 'bn_13', 'relu_14', 'conv_14', 'bn_14', 'relu_15', 'conv_15', 'permute_16', 'yolo_16', 'identity_17', 'conv_18', 'bn_18', 'relu_19', 'upsample_19', 'concat_20', 'conv_21', 'bn_21', 'relu_22', 'conv_22', 'permute_23', 'yolo_23']


In [35]:
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
    print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [36]:
while True:
    # 웹캠 프레임
    ret, frame = VideoSignal.read()
    # 이미지의 높이, 너비, 채널 받아오기
    h, w, c = frame.shape

    # yolo에 넣기 위한 전처리
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (224, 224), (0, 0, 0),True, crop=False)
    # 전처리된 blob 네트워크에 입력
    YOLO_net.setInput(blob)
    # 결과 받아오기
    outs = YOLO_net.forward(output_layers)

    # 각각의 데이터를 저장할 빈 리스트
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:

            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                # 탐지된 객체의 너비, 높이 및 중앙 좌표값 찾기
                center_x = int(detection[0] * w)
                center_y = int(detection[1] * h)
                dw = int(detection[2] * w)
                dh = int(detection[3] * h)
                
                # 객체의 사각형 테두리 중 좌상단 좌표값 찾기
                x = int(center_x - dw / 2)
                y = int(center_y - dh / 2)
                
                boxes.append([x, y, dw, dh])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Non Maximum Suppression (겹쳐있는 박스 중 confidence 가 가장 높은 박스를 선택)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.45, 0.4)


    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            score = confidences[i]
            color = colors[i]
            
            # 사각형 테두리 그리기 및 텍스트 쓰기
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 5)
            cv2.putText(frame, label, (x, y - 20), cv2.FONT_ITALIC, 0.5, 
            color, 1)

    # 함수를 이용해서 이미지를 모니터에 보여줌
    cv2.imshow("YOLOv3", frame)

    # 화면을 표시하고 키보드 입력을 대기하는 시간
    # 현재 1/1000ms를 기다려줌
    # 숫자 27은 아스키 코드로 ESC를 뜻하며
    # 실시간 웹캠 yolo을 종료하는 키
    
    if cv2.waitKey(1) == 27:
        break
# 객체 해제
VideoSignal.release()
# : 화면에 나타난 윈도우를 종료
cv2.destroyAllWindows()


KeyboardInterrupt: 